# KA SEIRSPlus
> backtesting on KA data

- toc: true 
- badges: false
- comments: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [2]:
# hide
# !pip install seirsplus
# !pip install -U plotly

import contextlib
import io
import json
import random
import sys
import warnings
from pathlib import Path
from typing import List, Union

import pandas as pd
# from branca.colormap import linear
from IPython.display import Latex, Markdown, display
from IPython.utils import io
from ipywidgets import (HTML, FloatLogSlider, FloatSlider, GridBox, HBox,
                        IntSlider, Label, Layout, Output, SelectionSlider,
                        VBox, interactive)
from seirsplus.models import *

warnings.filterwarnings("ignore")

In [6]:
#hide
ka_df = pd.read_csv("Karnataka_District_Info.csv")
# ka_df["Population"] = pd.to_numeric(ka_df["Population"].apply(lambda x: x.replace(",", "")))
ka_df

,District,CaseCount,BusCount,QTNCount
0,Kalaburagi,3,6.0,4
1,Kodagu,1,27.0,47
2,Chikkaballapura,4,0.0,9
3,Mysuru,3,19.0,231
4,Dharwad,1,12.0,39
5,Uttara Kannada,4,23.0,23
6,Dhakshina Kannada,5,82.0,756
7,Udupi,1,29.0,414
8,Tumkur,1,1.0,78
9,Shivamogga,0,43.0,99


In [0]:
# hide
def get_infections(
    initI: int = 100, initN: int = 10 ** 5, days_N: int = 21, beta: float = 2.4
) -> List[int]:
    model = SEIRSModel(
        beta=beta,
        sigma=1 / 5.2,
        gamma=1 / 12.39,
        initN=initN,
        initI=initI,
        beta_D=0.000,
        mu_D=0.02,
        sigma_D=1 / 4.0,
        gamma_D=1 / 9.0,
        theta_E=0.0002,
        theta_I=0.002,
        psi_E=0.2,
        psi_I=1.0,
    )
    with io.capture_output() as captured:
        model.run(T=days_N)
    S = model.numS  # time series of S counts
    E = model.numE  # time series of E counts
    I = model.numI  # time series of I counts
    D_E = model.numD_E  # time series of D_E counts
    D_I = model.numD_I  # time series of D_I counts
    R = model.numR  # time series of R counts
    F = model.numF  # time series of F counts
    t = model.tseries  # time values corresponding to the above time series
    return {"detected_exposed": D_E, "detected_infected": D_I, "model": model, "t": t}

In [0]:
# hide
def get_risk_estimates(
    assam_estimated_df: pd.DataFrame, beta: float
) -> Union[List, List]:
    days_N = 21
    atrisk_day14, atrisk_day21 = [], []
    for row in assam_estimated_df[["initI", "initN", "District"]].iterrows():
        initI, initN, district = row[1][0], int(row[1][1]), row[1][2]
        #     print(type(initI), type(initN))
        infection_results = get_infections(
            initI=initI, initN=initN, days_N=days_N, beta=beta
        )
        detected_infected = infection_results["detected_infected"]
        day14 = int(14 * len(detected_infected) / days_N)
        case_count_day14 = int(infection_results["detected_infected"][day14])
        case_count_day21 = int(infection_results["detected_infected"][-1])
        atrisk_day14.append(case_count_day14)
        atrisk_day21.append(case_count_day21)
    return atrisk_day14, atrisk_day21, infection_results

In [0]:
#hide_input
#@title Assumptions
percent_travellers_infected = 1e-1 #@param {type:"slider", min:0, max:1e-1, step:0.001}
unknown_to_known_travelers = 50 #@param {type:"slider", min:5, max:50, step:1}
beta = 3.0 #@param {type:"slider", min:0.5, max:5.0, step:0.1}

#display(Markdown(f"#Assumptions \npercent_travellers_infected :{percent_travellers_infected}\nunknown_to_known_travelers={unknown_to_known_travelers}\nbeta:{beta} "))

In [0]:
# hide
def estimate(
    percent_travellers_infected: float,
    unknown_to_known_travelers: int,
    beta: float,
    state_df: pd.DataFrame,
):

    state_estimated_df = state_df.copy()
    state_estimated_df["initI"] = (
        percent_travellers_infected * unknown_to_known_travelers * state_df["Inflow"]
    ).astype(int)
    state_estimated_df["initN"] = state_estimated_df["Population"].astype(int)

    (
        state_estimated_df[f"day14"],
        state_estimated_df[f"day21"],
        infection_results,
    ) = get_risk_estimates(state_estimated_df, beta=beta)
    return state_estimated_df

In [0]:
# hide
ka_estimated_df = estimate(
    percent_travellers_infected, unknown_to_known_travelers, beta, ka_df
)

In [0]:
# hide
ka_estimated_df["Day 21 Infections"] = ka_estimated_df["day21"]
ka_estimated_df["Day 14 Infections"] = ka_estimated_df["day14"]

In [77]:
# hide
ka_estimated_df["text"] = (
    "District : "
    + ka_estimated_df["District"].astype(str)
    + "<br>"
    + "Population: "
    + ka_estimated_df["Population"].astype(str)
    + "<br>"
    + "Day 14 Infections : "
    + ka_estimated_df["Day 14 Infections"].astype(str)
    + "<br>"
    + "Day 21 Infections : "
    + ka_estimated_df["Day 21 Infections"].astype(str)
    + "<br>"
    + "Travellers: "
    + ka_estimated_df["Inflow"].astype(str)
)

ka_estimated_df.head()

,District,Population,Inflow,initI,initN,day14,day21,Day 21 Infections,Day 14 Infections,text
0,Bagalkot,1889752,26,130,1889752,4097,11647,11647,4097,District : Bagalkot<br>Population: 1889752<br>...
1,Bangalore,9621551,7315,36575,9621551,49826,68475,68475,49826,District : Bangalore<br>Population: 9621551<br...
2,Belgaum,4779661,16,80,4779661,5457,26105,26105,5457,District : Belgaum<br>Population: 4779661<br>D...
3,Bellary,2452595,44,220,2452595,5835,15385,15385,5835,District : Bellary<br>Population: 2452595<br>D...
4,Bidar,1703300,0,0,1703300,0,0,0,0,District : Bidar<br>Population: 1703300<br>Day...


# District vs AtRisk

In [78]:
# hide_input
ka_estimated_df[["District", "Day 14 Infections", "Day 21 Infections"]]

,District,Day 14 Infections,Day 21 Infections
0,Bagalkot,4097,11647
1,Bangalore,49826,68475
2,Belgaum,5457,26105
3,Bellary,5835,15385
4,Bidar,0,0
5,Bijapur,0,0
6,Chamarajanagar,1554,5885
7,Chikkaballapura,2093,7361
8,Chikmagalur,2963,7260
9,Chitradurga,0,0
